In [2]:
# 실제 train data를 변환하기 - 데이터 읽어들이기

import json
import os
import pandas as pd
import numpy as np

base_dir = os.getcwd()

train_images_path = base_dir + '/data/train/images/'
train_labels_json_path = base_dir + '/data/train/labels_json/'
train_labels_path = base_dir + '/data/train/labels/'

os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_json_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)

!unzip -qq '/content/drive/MyDrive/KCC3기_2차_프로젝트/train.zip' -d '/content/data/train'

train_images_list = os.listdir(train_images_path)
train_labels_json_list = os.listdir(train_labels_json_path)

train_images_list.sort()
train_labels_json_list.sort()

In [3]:
# 실제 train data를 변환하기 - 라벨링 변환

yolo_class = {
    '철캔': 0,
    '알루미늄캔': 1,
    '종이': 2,
    '무색단일': 3,
    '유색단일': 4,
    'PE': 5,
    'PP': 6,
    'PS': 7,
    '스티로폼': 8,
    '비닐': 9,
    '갈색': 10,
    '녹색': 11,
    '투명': 12
}

for label in train_labels_json_list:
  file_json = open(train_labels_json_path + label)
  f_json = json.load(file_json)

  img_width = f_json['IMAGE_INFO']['IMAGE_WIDTH']
  img_height = f_json['IMAGE_INFO']['IMAGE_HEIGHT']

  f_txt = open(train_labels_path + label.replace('.json', '.txt'), 'w')

  for anno in f_json['ANNOTATION_INFO']:
    obj_id = yolo_class[anno['DETAILS']]
    obj_points = np.array(anno['POINTS'])
    if obj_points[0].size == 4:
      yolo_id = obj_id
      yolo_points = np.array([obj_points[0][0]+obj_points[0][2]/2, obj_points[0][1]+obj_points[0][3]/2,
                              obj_points[0][2], obj_points[0][3]])

      yolo_points_norm = np.array([yolo_points[0]/img_width, yolo_points[1]/img_height,
                             yolo_points[2]/img_width, yolo_points[3]/img_height])

      yolo_points_norm = list(yolo_points_norm)
      f_txt.write(str(yolo_id) + ' ')
      f_txt.write(' '.join(str(ele) for ele in yolo_points_norm))
      f_txt.write('\n')
    elif obj_points.size >= 2:
      max_x = 0
      min_x = img_width
      max_y = 0
      min_y = img_height
      for points in obj_points:
        if points[0] > max_x:
          max_x = points[0]
        if points[0] < min_x:
          min_x = points[0]
        if points[1] > max_y:
          max_y = points[1]
        if points[1] < min_y:
          min_y = points[1]

      yolo_id = obj_id
      yolo_points = np.array([(min_x+max_x)/2, (min_y+max_y)/2,
                              max_x-min_x, max_y-min_y])
      yolo_points_norm = np.array([yolo_points[0]/img_width, yolo_points[1]/img_height,
                             yolo_points[2]/img_width, yolo_points[3]/img_height])
      yolo_points_norm = list(yolo_points_norm)

      f_txt.write(str(yolo_id) + ' ')
      f_txt.write(' '.join(str(ele) for ele in yolo_points_norm))
      f_txt.write('\n')
    else:
      print(label)
  f_txt.close()


6 0.4915269230769231 0.49103166345617255 0.38315604395604386 0.6721641025641029

0 0.6400563522733375 0.3877540579713878 0.1944409271364939 0.387188312642922



In [ ]:
# json 라벨 삭제 및 라벨링 확인

import shutil

shutil.rmtree('/content/data/train/labels_json')

print(open('/content/data/train/labels/606511@4_04002_220907_P1_T1.txt').readline())
print(open('/content/data/train/labels/10349@3_01001_220715_P1_T1.txt').readline())

In [4]:
# import shutil

# shutil.move('/content/data/train/labels' , '/content/drive/MyDrive/KCC3기_2차_프로젝트/train/')

'/content/drive/MyDrive/KCC3기_2차_프로젝트/train/labels/labels'

In [1]:
# !zip -r "/content/data/train.zip" "/content/data/train/"

In [5]:
import shutil

val_images_path = base_dir + '/data/val/images/'
val_labels_path = base_dir + '/data/val/labels/'

os.makedirs(val_images_path, exist_ok=True)
os.makedirs(val_labels_path, exist_ok=True)

train_images_list = os.listdir(train_images_path)
train_labels_list = os.listdir(train_labels_path)

train_images_list.sort()
train_labels_list.sort()

for i in range(2600):
  shutil.move(train_images_path + train_images_list[i*10], val_images_path)
  shutil.move(train_labels_path + train_labels_list[i*10], val_labels_path)


In [10]:
print(len(os.listdir('/content/data/train/images/')))
print(len(os.listdir('/content/data/train/labels/')))

print(len(os.listdir('/content/data/val/images/')))
print(len(os.listdir('/content/data/val/labels/')))

train_images_list = os.listdir(train_images_path)
train_labels_list = os.listdir(train_labels_path)

train_images_list.sort()
train_labels_list.sort()

for i in range(len(train_images_list)):
  if train_images_list[i].replace('.jpg', '') != train_labels_list[i].replace('.txt', ''):
    print(train_images_list[i])

val_images_list = os.listdir(val_images_path)
val_labels_list = os.listdir(val_labels_path)

val_images_list.sort()
val_labels_list.sort()

for i in range(len(val_images_list)):
  if val_images_list[i].replace('.jpg', '') != val_labels_list[i].replace('.txt', ''):
    print(val_images_list[i])


23400
23400
2600
2600


In [3]:
!pip install PyYAML

import yaml

data = {
    'train': '/content/data/train/images/',
    'val': '/content/data/val/images',
    'names': ['IronCan', 'AluminumCan', 'Paper', 'PetNoColor',
              'PetColor', 'PE', 'PP', 'PS', 'Styrofoam',
              'Vinyl', 'GlassBrown', 'GlassGreen', 'GlassTransparency'],
    'nc': 13
}

yaml_name = 'recycle_data.yaml'
with open('/content/data/' + 'recycle_data.yaml', 'w') as f:
  yaml.dump(data, f)

with open('/content/data/' + 'recycle_data.yaml', 'r') as f:
  recycle_yaml = yaml.safe_load(f)
  display(recycle_yaml)

{'names': ['IronCan',
  'AluminumCan',
  'Paper',
  'PetNoColor',
  'PetColor',
  'PE',
  'PP',
  'PS',
  'Styrofoam',
  'Vinyl',
  'GlassBrown',
  'GlassGreen',
  'GlassTransparency'],
 'nc': 13,
 'train': '/content/data/train/images/',
 'val': '/content/data/val/images'}

In [13]:
# Setup module

%pip install ultralytics

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.155 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 36.5/78.2 GB disk)


In [16]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.train(data='/content/data/recycle_data.yaml', epochs=5, patience=3, batch=64, imgsz=640, save_dir='/content/drive/MyDrive/KCC3기_2차_프로젝트/models/train3')

Ultralytics YOLOv8.0.155 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data/recycle_data.yaml, epochs=5, patience=3, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=Fa

In [1]:
import shutil

shutil.move('/content/runs/detect/result/' , '/content/drive/MyDrive/KCC_2nd_project/models/')


'/content/drive/MyDrive/KCC_2nd_project/models/result'

In [5]:
from ultralytics import YOLO
import shutil, os

model = YOLO('/content/drive/MyDrive/KCC_2nd_project/models/result/weights/best.pt')

model.train(data='/content/data/recycle_data.yaml', epochs=3, patience=3, batch=64, imgsz=640, name='result', exist_ok=True)

if os.path.isdir('/content/drive/MyDrive/KCC_2nd_project/models/result'):
  shutil.rmtree('/content/drive/MyDrive/KCC_2nd_project/models/result')
  shutil.move('/content/runs/detect/result/' , '/content/drive/MyDrive/KCC_2nd_project/models/')

Ultralytics YOLOv8.0.155 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/KCC_2nd_project/models/result/weights/best.pt, data=/content/data/recycle_data.yaml, epochs=3, patience=3, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=result, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=Tru

In [15]:
from PIL import Image
import matplotlib.pyplot as plt

results = model('/content/data/val/images/' + os.listdir('/content/data/val/images/')[0])

# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    print(im)
    im.show()  # show image
    # im.save('results.jpg')


image 1/1 /content/data/val/images/10817@5_01001_220721_P1_T1.jpg: 640x640 1 IronCan, 17.1ms
Speed: 10.1ms preprocess, 17.1ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)


<PIL.Image.Image image mode=RGB size=640x640 at 0x7B8BAD74EB30>
